In [4]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import os
from ultralytics import YOLO
import chess
from stockfish import Stockfish
import time
import threading

In [3]:
# Variable to toggle
ch = 0

def toggle_ch():
    global ch
    while True:
        ch = 1
        time.sleep(10)

# Start the toggle function in a separate thread
toggle_thread = threading.Thread(target=toggle_ch)
toggle_thread.start()

# Load the YOLO model
model_path = os.path.join('.', 'runs', 'segment', 'train_CB', 'weights', 'best.pt')
model = YOLO(model_path)

model_path1 = os.path.join('.', 'runs', 'detect', 'train_GCP', 'weights', 'best.pt')
model1 = YOLO(model_path1)

video_path = "C:/Users/Daithoulung Rongmai/Pictures/Camera Roll/WIN_20241022_22_59_02_Pro.mp4"  # Replace with your video path
video = cv2.VideoCapture(video_path)

# Check if the video is opened correctly
if not video.isOpened():
    print("Error: Could not open video stream.")
    exit()

stockfish=Stockfish("StockFish/stockfish-windows-x86-64-avx2.exe")
stockfish.set_depth(20)
stockfish.set_skill_level(20)
stockfish.get_parameters()
board = chess.Board(None) 
board.turn = chess.BLACK

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Run YOLO inference on the current frame
    results = model.predict(frame, conf=0.8)

    # Initialize variable to check if warping happened
    warping_done = False

    for result in results:
        # Check if masks are available and not None
        if hasattr(result, 'masks') and result.masks is not None:
            masks = result.masks.data.cpu().numpy()  # Extract the mask data

            for i in range(masks.shape[0]):
                mask = masks[i]

                # Convert mask to binary (0, 255)
                mask_binary = (mask * 255).astype(np.uint8)

                # Resize mask to match the frame size
                mask_binary_resized = cv2.resize(mask_binary, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)

                # Extract the masked image
                masked_frame = cv2.bitwise_and(frame, frame, mask=mask_binary_resized)

                # Find contours of the mask
                contours, _ = cv2.findContours(mask_binary_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # If contours are found
                if contours:
                    largest_contour = max(contours, key=cv2.contourArea)

                    # Approximate the contour to get corners
                    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
                    approx_corners = cv2.approxPolyDP(largest_contour, epsilon, True)

                    if len(approx_corners) == 4:  # Need 4 corners for perspective transformation
                        src_pts = np.array([pt[0] for pt in approx_corners], dtype="float32")

                        # Destination points for the top-down view (600x600)
                        dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")

                        # Compute perspective transform matrix
                        M = cv2.getPerspectiveTransform(src_pts, dst_pts)

                        # Apply warp perspective transformation
                        warped_frame = cv2.warpPerspective(masked_frame, M, (600, 600))

                        # Optionally rotate and flip the image for correct orientation
                        rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                        mirrored_frame = cv2.flip(rotated_frame, 1)

                        # Create a grid on the warped image
                        grid_size = 600
                        rows, cols = 8, 8
                        cell_width = grid_size // cols
                        cell_height = grid_size // rows

                        # Draw horizontal and vertical grid lines
                        img_with_grid = mirrored_frame.copy()
                        for r in range(1, rows):
                            y = r * cell_height
                            cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                        for c in range(1, cols):
                            x = c * cell_width
                            cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)

                        # Display the warped mask with grid
                        # cv2.imshow(f"Warped Mask {i + 1}", img_with_grid)
                        warping_done = True  # Indicate that warping has been done using a mask
                    else:
                        print("Could not find 4 corners for perspective transformation.")
                else:
                    print("No contours found for the mask.")
        else:
            print("No masks found in the results.")

    # If no mask was used for warping, try using the bounding box of the chessboard
    if not warping_done:
        for result in results:
            if hasattr(result, 'boxes') and result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
                if len(boxes) > 0:
                    # Use the first detected bounding box for the chessboard
                    x_min, y_min, x_max, y_max = boxes[0]
                    src_pts = np.array([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]], dtype="float32")

                    # Destination points for top-down view (600x600)
                    dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")

                    # Compute perspective transform matrix
                    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

                    # Apply warp perspective transformation
                    warped_frame = cv2.warpPerspective(frame, M, (600, 600))

                    # Optionally rotate and flip the image for correct orientation
                    rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                    mirrored_frame = cv2.flip(rotated_frame, 1)

                    # Create a grid on the warped image
                    grid_size = 600
                    rows, cols = 8, 8
                    cell_width = grid_size // cols
                    cell_height = grid_size // rows

                    # Draw horizontal and vertical grid lines
                    img_with_grid = mirrored_frame.copy()
                    for r in range(1, rows):
                        y = r * cell_height
                        cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                    for c in range(1, cols):
                        x = c * cell_width
                        cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)

                    # Display the warped frame with grid
                    # cv2.imshow("Warped Chessboard", img_with_grid)
                    warping_done = True
                    break  # No need to check further bounding boxes once warping is done
    columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    rows_labels = ['1', '2', '3', '4', '5', '6', '7', '8']
    
    # Reverse the row labels to make sure bottom-left is 'a1'
    rows_labels.reverse()
    
    # Copy the warped image to draw labels
    img_with_labels = img_with_grid.copy()
    
    # Iterate over each cell in the grid
    for i in range(rows):
        for j in range(cols):
            # Calculate the top-left corner of the current cell
            x = i * cell_width
            y = j * cell_height
    
            # Generate the corresponding chessboard label
            label = columns[i] + rows_labels[j]
    
            # Place the label at the center of the cell
            text_x = x + cell_width // 2 - 15  # Adjust for text alignment
            text_y = y + cell_height // 2 + 15  # Adjust for text alignment
    
            # Add the label to the image (white text with font size 1)
            cv2.putText(img_with_labels, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    res = img_with_labels.copy()

    piece_positions = {}
    
    results1 = model1.predict(mirrored_frame, conf=0.8)
    
    for result in results1:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        names = result.names
        for box,class_id in zip(boxes,classes):
            x_min,y_min,x_max,y_max = box
            center_x = (x_min + x_max) / 2
            center_y = ((y_min + y_max) / 2)
            
            grid_col = int(center_x // cell_width)
            grid_row = int(center_y // cell_height)
            
            if 0 <= grid_col < cols and 0 <= grid_row < rows:
                # Get the corresponding grid label
                grid_label = columns[grid_col] + rows_labels[grid_row]
    
                # Store the piece position and type in the dictionary
                piece_positions[grid_label] = names[int(class_id)]  # Correctly map label to class name
                
                cv2.circle(res, (int(center_x), int(center_y)), 5, (255, 0, 0), -1)
                cv2.putText(res, grid_label, (int(center_x)-10, int(center_y)-15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    piece_map = {
        'white-rook': chess.Piece(chess.ROOK, chess.WHITE),
        'white-knight': chess.Piece(chess.KNIGHT, chess.WHITE),
        'white-bishop': chess.Piece(chess.BISHOP, chess.WHITE),
        'white-queen': chess.Piece(chess.QUEEN, chess.WHITE),
        'white-king': chess.Piece(chess.KING, chess.WHITE),
        'white-pawn': chess.Piece(chess.PAWN, chess.WHITE),
        'black-rook': chess.Piece(chess.ROOK, chess.BLACK),
        'black-knight': chess.Piece(chess.KNIGHT, chess.BLACK),
        'black-bishop': chess.Piece(chess.BISHOP, chess.BLACK),
        'black-queen': chess.Piece(chess.QUEEN, chess.BLACK),
        'black-king': chess.Piece(chess.KING, chess.BLACK),
        'black-pawn': chess.Piece(chess.PAWN, chess.BLACK),
    }
    
    def set_up_board(board, piece_positions):
        for position, piece_description in piece_positions.items():
            piece = piece_map[piece_description]  
            square = chess.parse_square(position) 
            board.set_piece_at(square, piece)

    set_up_board(board, piece_positions)
    
    fen_string = board.fen()
    
    board = chess.Board(fen_string)

    if ch==1:
        if (board.turn == chess.BLACK):
            board.turn = chess.WHITE
        else:
            board.turn = chess.BLACK
            
        stockfish.set_fen_position(board.fen())
        top_moves = stockfish.get_top_moves(3)
        ch=0

    img_with_Ans = img_with_grid.copy()
    for i in range(rows):
        for j in range(cols):
            # Calculate the top-left corner of the current cell
            x = i * cell_width
            y = j * cell_height
    
            # Generate the corresponding chessboard label
            label = columns[i] + rows_labels[j]
    
            # Place the label at the center of the cell
            text_x = x + cell_width // 2 - 10  # Adjust for text alignment
            text_y = y + cell_height // 2 + 10  # Adjust for text alignment
            C = 0
            for move in top_moves:
                if label == (move['Move'][2:]):
                    cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (C+100, C, 255-C), 2)
                if label == (move['Move'][:2]):
                    cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (C+100, C, 255-C), 2)
                    cv2.circle(img_with_Ans, (int(text_x), int(text_y)), 5, (0, 255, 255), -1)
                C+=70
    
    cv2.imshow("Chessboard", img_with_Ans)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(2)

# Release the video capture and close windows
video.release()
cv2.destroyAllWindows()



0: 480x800 1 0, 40.1ms
Speed: 7.0ms preprocess, 40.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 800x800 1 black-king, 1 black-knight, 6 black-pawns, 1 black-queen, 1 black-rook, 2 white-bishops, 1 white-king, 4 white-pawns, 1 white-queen, 11.1ms
Speed: 5.0ms preprocess, 11.1ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 0, 32.5ms
Speed: 4.0ms preprocess, 32.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 800x800 1 black-king, 1 black-knight, 6 black-pawns, 1 black-queen, 1 black-rook, 2 white-bishops, 1 white-king, 4 white-pawns, 1 white-queen, 35.1ms
Speed: 5.0ms preprocess, 35.1ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 800)
